In [1]:

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from utils import *
from train import tf_dataset
from metrics import iou


In [2]:

def read_image(x):
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = np.clip(image - np.median(image)+127, 0, 255)
    image = image/255.0
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)
    return image

def read_mask(y):
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = mask.astype(np.float32)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=-1)
    return mask

def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def parse(y_pred):
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = y_pred[..., -1]
    y_pred = y_pred.astype(np.float32)
    y_pred = np.expand_dims(y_pred, axis=-1)
    return y_pred

m = tf.keras.metrics.MeanIoU(num_classes=2)
iou_ary = []
dice_coef_ary=[]
def evaluate_normal(model, x_data, y_data):
    THRESHOLD = 0.5
    total = []
    for i, (x, y) in tqdm(enumerate(zip(x_data, y_data)), total=len(x_data)):
        x = read_image(x)
        y = read_mask(y)
        _, h, w, _ = x.shape

        #y_pred1 = parse(model.predict(x)[0][..., -2])
        #y_pred2 = parse(model.predict(x)[0][..., -1])
        y_pred3 = parse(model.predict(x)[0][..., -1])
        y_pred3[y_pred3<THRESHOLD] = 0
        y_pred3[y_pred3>THRESHOLD] = 1
        m.reset_states()
        m.update_state(y, y_pred3)
        #print("y",y)
        #print("y_pred3",y_pred3)
        #print("num ",i," iou: ",m.result().numpy())
        #iou_ary.append(m.result().numpy())
        iou_ary.append(iou(y,y_pred3))
        dice_coef_ary.append(dice_coef(y,y_pred3))

        line = np.ones((h, 10, 3)) * 255.0
        
        all_images = [
            x[0] * 255.0, line,
            mask_to_3d(y) * 255.0, line,
            mask_to_3d(y_pred3) * 255.0
        ]
        mask = np.concatenate(all_images, axis=1)

        cv2.imwrite(f"results/{i}.png", mask)

smooth = 1
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)
    create_dir("results/")

    batch_size = 4

    test_path = "data_comb1/test/"
    test_x = sorted(glob(os.path.join(test_path, "img", "*.png")))
    test_y = sorted(glob(os.path.join(test_path, "masks_machine", "*.png")))
    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    test_steps = (len(test_x)//batch_size)
    if len(test_x) % batch_size != 0:
        test_steps += 1
    print(test_steps)
    model = load_model_weight("5videos_20_da5.h5")
    model.evaluate(test_dataset, steps=test_steps)
    
    evaluate_normal(model, test_x, test_y)
    print("iou_ary mean  ",np.mean(iou_ary))
    print("dice_coef mean  ",np.mean(dice_coef_ary))

563


OSError: SavedModel file does not exist at: 5videos_20_da5.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
import  scipy.stats as st
print(np.mean(iou_ary))

print(st.sem(iou_ary))

st.t.interval(0.95, len(iou_ary)-1, loc=np.mean(iou_ary), scale=st.sem(iou_ary))



In [ ]:
st.t.interval(0.95, len(dice_coef_ary)-1, loc=np.mean(dice_coef_ary), scale=st.sem(dice_coef_ary))
